In [24]:
import os
import numpy as np
import datetime as dt
os.chdir("../")
from datasets import load_dataset, Dataset, DatasetDict
from src.data.processing import DataProcessing

In [2]:
dataset = load_dataset("monash_tsf", "traffic_hourly")
freq = "1H"

Found cached dataset monash_tsf (/home/onyxia/.cache/huggingface/datasets/monash_tsf/traffic_hourly/1.0.0/82998723d55d6edbc664cfaebe371004d849846ee5f61b57f1f974000d44c050)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data_pipeline = DataProcessing(dataset)

In [4]:
mtrain, mtest = data_pipeline.multi_variate_format(freq)

In [25]:
ds = Dataset.from_dict({'target': [1, 2, 3], 'feat_static_cat': [0, 0, 0], 'feat_dynamic_real': [None, None, None], 'item_id': ["T1", "T1", "T1"]})
DatasetDict({"train": ds})

DatasetDict({
    train: Dataset({
        features: ['target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'],
        num_rows: 3
    })
})

In [8]:
ds

NameError: name 'ds' is not defined

In [23]:
dataset["train"][0]

{'start': Period('2015-01-01 00:00', 'H'),
 'target': [0.004800000227987766,
  0.007199999876320362,
  0.004000000189989805,
  0.0038999998942017555,
  0.00419999985024333,
  0.004000000189989805,
  0.004399999976158142,
  0.00419999985024333,
  0.00860000029206276,
  0.022299999371170998,
  0.03269999846816063,
  0.04089999943971634,
  0.045099999755620956,
  0.04619999974966049,
  0.049800001084804535,
  0.043699998408555984,
  0.043699998408555984,
  0.040699999779462814,
  0.037700001150369644,
  0.029500000178813934,
  0.02590000070631504,
  0.01360000018030405,
  0.008200000040233135,
  0.0038999998942017555,
  0.0038999998942017555,
  0.004000000189989805,
  0.003800000064074993,
  0.003700000001117587,
  0.004100000020116568,
  0.022600000724196434,
  0.042899999767541885,
  0.0421999990940094,
  0.04529999941587448,
  0.05490000173449516,
  0.05770000070333481,
  0.061400000005960464,
  0.0658000037074089,
  0.06300000101327896,
  0.06520000100135803,
  0.0649000033736229,
  0